## Ide feature extraction

1. floors_before_eq (total) -> Structured parse pake LLM
2. old_building -> standarisasi?
3. plinth_area -> Structured parse pake LLM
4. height_before_eq -> standarisasi
5. land_surface_condition -> OHE
6. type_of_foundation -> OHE
7. type_of_roof -> OHE
8. type_of_ground_floor -> OHE
9. type_of_other_floor -> OHE
10. position -> OHE
11. building_plan_configuration -> OHE
12. technical_solution_proposed -> OHE (aku ga ngerti apa ada pengaruhnya fitur ini dengan damage_grade)
13. legal_ownership_status -> benerin dulu, LE
14. has_secondary_use -> 
15. type_of_reinforcement_concrete -> OHE
16. residential_type -> OHE
17. no_family_residing -> OHE (aku ga ngerti apa ada pengaruhnya fitur ini dengan damage_grade)
18. public_place_type -> OHE
19. industrial_use_type -> OHE
20. govermental_use_type -> OHE
21. flexible_superstructure -> OHE
22. wall_binding -> OHE
23. wall_material -> OHE
24. damage_grade -> 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI

warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv("../dataset/raw/train.csv", )
train.drop(["Unnamed: 0"], axis=1, inplace=True)
train.head()

,floors_before_eq (total),old_building,plinth_area (ft^2),height_before_eq (ft),land_surface_condition,type_of_foundation,type_of_roof,type_of_ground_floor,type_of_other_floor,position,...,type_of_reinforcement_concrete,residential_type,no_family_residing,public_place_type,industrial_use_type,govermental_use_type,flexible_superstructure,wall_binding,wall_material,damage_grade
0,floor two,1.0,256 ft^2,22.0,Flat,Bamboo or Timber,Bamboo/Timber Light roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,0.0,0.0,1.0
1,Floor 3,3.0,985 ft^2,18.0,Flat,Clay Sand Mixed mortar-Stone/Brick,Wood Light Roof or Bamboo Heavy Roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
2,Two Floor,7.0,NaN,14.0,Flat,Mud mortar-Stone/Brick,NaN,Clay,Wood-Mud or Bamboo Mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
3,two,18.0,185 ft^2,15.0,Flat,Clay Sand Mixed mortar-Stone/Brick,Wood Light Roof or Bamboo Light Roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,4.0
4,just 2 floor,22.0,290 ft^2,17.0,Flat,Clay Sand Mixed mortar-Stone/Brick,Bamboo or Timber Light roof,Clay,Timber Mud or Bamboo-Mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,1.0


In [3]:
# Gajadi
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_KEY")

class FloorB4Eq(BaseModel):
    floor: int = Field(description="Number of floor of the building")

parser = PydanticOutputParser(pydantic_object=FloorB4Eq)

model_name = 'gpt-3.5-turbo'
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature,
               openai_api_key=OPENAI_API_KEY)


prompt = PromptTemplate(
    template="{format_instructions}\nNumber of floor in given context:\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)


def floor_from_context(context):
    if context is "" or None:
        return np.nan
    else:
        _input = prompt.format_prompt(query=context)
        output = model(_input.to_string())
        return parser.parse(output).floor


In [3]:
print(train.shape)
train.dropna(inplace=True)
print(train.shape)

(722815, 24)
(46801, 24)


In [4]:
train.reset_index(inplace=True)
train.drop(["index"], axis=1, inplace=True)

In [5]:
train.head()

,floors_before_eq (total),old_building,plinth_area (ft^2),height_before_eq (ft),land_surface_condition,type_of_foundation,type_of_roof,type_of_ground_floor,type_of_other_floor,position,...,type_of_reinforcement_concrete,residential_type,no_family_residing,public_place_type,industrial_use_type,govermental_use_type,flexible_superstructure,wall_binding,wall_material,damage_grade
0,Has 1 floor,22.0,300 ft^2,10.0,Moderate slope,Clay Sand Mixed mortar-Stone/Brick,Bamboo/Timber Light roof,mud,Not applicable,Not attached,...,0.0,Non-residential,None,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
1,Three floor,22.0,360 ft^2,18.0,Flat,Clay mortar-Stone/Brick,Wood Light Roof or Bamboo Heavy Roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,4.0
2,Floor 2,8.0,380 ft^2,17.0,Flat,Clay mortar-Stone/Brick,Wood Light Roof or Bamboo Light Roof,Brick or Stone,Timber-Planck,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
3,two,22.0,250 ft^2,14.0,Flat,Clay mortar-Stone/Brick,Bamboo/Timber Light roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,None,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
4,Floor two,7.0,312 ft^2,13.0,Flat,Clay Sand Mixed mortar-Stone/Brick,Bamboo or Timber Light roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,available,5.0,2.0,2.0


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46801 entries, 0 to 46800
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   floors_before_eq (total)        46801 non-null  object 
 1   old_building                    46801 non-null  float64
 2   plinth_area (ft^2)              46801 non-null  object 
 3   height_before_eq (ft)           46801 non-null  float64
 4   land_surface_condition          46801 non-null  object 
 5   type_of_foundation              46801 non-null  object 
 6   type_of_roof                    46801 non-null  object 
 7   type_of_ground_floor            46801 non-null  object 
 8   type_of_other_floor             46801 non-null  object 
 9   position                        46801 non-null  object 
 10  building_plan_configuration     46801 non-null  object 
 11  technical_solution_proposed     46801 non-null  object 
 12  legal_ownership_status          

In [7]:
train_cp = train.copy()
angka = {
    "one": 1,
    "two": 2,
    "three": 3,
    "four": 4,
    "five": 5,
    "six": 6,
    "seven": 7,
    "eight": 8,
    "nine": 9,
    "1st": 1,
    "2nd": 2,
    "3rd": 3,
    "fifth": 5,
    "second": 2,
    "third": 3,
    "3.00": 3,
}

delete = {
    "story": "",
    "stories": "",
    "floor": "",
    "floors": "",
    "has": "",
    "there is": "",
    "just": "",
    "-": "",
    "fl": "",
    "/": "",
}

train_cp["floors_before_eq (total)"] = train_cp["floors_before_eq (total)"]\
                                            .str.lower()\
                                            .replace(delete, regex=True)\
                                            .str.strip()\
                                            .replace(angka, regex=True)

train_cp["floors_before_eq (total)"] = train_cp["floors_before_eq (total)"].astype(
    "float")
train_cp.head()

,floors_before_eq (total),old_building,plinth_area (ft^2),height_before_eq (ft),land_surface_condition,type_of_foundation,type_of_roof,type_of_ground_floor,type_of_other_floor,position,...,type_of_reinforcement_concrete,residential_type,no_family_residing,public_place_type,industrial_use_type,govermental_use_type,flexible_superstructure,wall_binding,wall_material,damage_grade
0,1.0,22.0,300 ft^2,10.0,Moderate slope,Clay Sand Mixed mortar-Stone/Brick,Bamboo/Timber Light roof,mud,Not applicable,Not attached,...,0.0,Non-residential,None,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
1,3.0,22.0,360 ft^2,18.0,Flat,Clay mortar-Stone/Brick,Wood Light Roof or Bamboo Heavy Roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,4.0
2,2.0,8.0,380 ft^2,17.0,Flat,Clay mortar-Stone/Brick,Wood Light Roof or Bamboo Light Roof,Brick or Stone,Timber-Planck,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
3,2.0,22.0,250 ft^2,14.0,Flat,Clay mortar-Stone/Brick,Bamboo/Timber Light roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,None,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
4,2.0,7.0,312 ft^2,13.0,Flat,Clay Sand Mixed mortar-Stone/Brick,Bamboo or Timber Light roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,available,5.0,2.0,2.0


In [8]:
train_cp["plinth_area (ft^2)"] = train_cp["plinth_area (ft^2)"].str.lower()
train_cp["plinth_area (ft^2)"] = train_cp["plinth_area (ft^2)"].apply(
    lambda x: x.replace(" ft^2", ""))
train_cp["plinth_area (ft^2)"] = train_cp["plinth_area (ft^2)"].apply(
    lambda x: x.replace("more than ", ""))
train_cp["plinth_area (ft^2)"] = train_cp["plinth_area (ft^2)"].astype("float")


In [9]:
land_surface_condition_dict = {
    "Steep slope" : 0,
    "Moderate slope": 1,
    "Flat": 2,
}

train_cp["land_surface_condition"] = train_cp["land_surface_condition"].replace(land_surface_condition_dict)
train_cp.head()

,floors_before_eq (total),old_building,plinth_area (ft^2),height_before_eq (ft),land_surface_condition,type_of_foundation,type_of_roof,type_of_ground_floor,type_of_other_floor,position,...,type_of_reinforcement_concrete,residential_type,no_family_residing,public_place_type,industrial_use_type,govermental_use_type,flexible_superstructure,wall_binding,wall_material,damage_grade
0,1.0,22.0,300.0,10.0,1,Clay Sand Mixed mortar-Stone/Brick,Bamboo/Timber Light roof,mud,Not applicable,Not attached,...,0.0,Non-residential,None,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
1,3.0,22.0,360.0,18.0,2,Clay mortar-Stone/Brick,Wood Light Roof or Bamboo Heavy Roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,4.0
2,2.0,8.0,380.0,17.0,2,Clay mortar-Stone/Brick,Wood Light Roof or Bamboo Light Roof,Brick or Stone,Timber-Planck,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
3,2.0,22.0,250.0,14.0,2,Clay mortar-Stone/Brick,Bamboo/Timber Light roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,None,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
4,2.0,7.0,312.0,13.0,2,Clay Sand Mixed mortar-Stone/Brick,Bamboo or Timber Light roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,available,5.0,2.0,2.0


In [10]:
type_of_foundation_dict = {
    "Mud mortar-Stone/Brick": "Clay mortar-Stone/Brick",
    "Bamboo/TImber": "Bamboo or Timber",
    "Bamboo/Timber": "Bamboo or Timber",
    "RC": "Reinforced Concrete",
    "Others": "Other",
    "Cement-Stone or Cement-Brick": "Cement-Stone/Brick"
}

train_cp["type_of_foundation"] = train_cp["type_of_foundation"]\
                                    .replace(type_of_foundation_dict)\
                                    .str.strip()

train_cp.head()

,floors_before_eq (total),old_building,plinth_area (ft^2),height_before_eq (ft),land_surface_condition,type_of_foundation,type_of_roof,type_of_ground_floor,type_of_other_floor,position,...,type_of_reinforcement_concrete,residential_type,no_family_residing,public_place_type,industrial_use_type,govermental_use_type,flexible_superstructure,wall_binding,wall_material,damage_grade
0,1.0,22.0,300.0,10.0,1,Clay Sand Mixed mortar-Stone/Brick,Bamboo/Timber Light roof,mud,Not applicable,Not attached,...,0.0,Non-residential,None,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
1,3.0,22.0,360.0,18.0,2,Clay mortar-Stone/Brick,Wood Light Roof or Bamboo Heavy Roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,4.0
2,2.0,8.0,380.0,17.0,2,Clay mortar-Stone/Brick,Wood Light Roof or Bamboo Light Roof,Brick or Stone,Timber-Planck,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
3,2.0,22.0,250.0,14.0,2,Clay mortar-Stone/Brick,Bamboo/Timber Light roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,None,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
4,2.0,7.0,312.0,13.0,2,Clay Sand Mixed mortar-Stone/Brick,Bamboo or Timber Light roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,available,5.0,2.0,2.0


In [11]:
type_of_roof_dict = {
    "Bamboo/TImber-Heavy Roof": "Bamboo/Timber Heavy roof",
    "Bamboo/TImber-Light Roof": "Bamboo or Timber Light roof",
    "Bamboo/Timber Light roof": "Bamboo or Timber Light roof ",
    "reinforced cement concrete/rb/rbc": "rcc/rb/rbc",
    "Reinforced brick concrete/rcc/rbc": "rcc/rb/rbc",
    "Bamboo or Timber Heavy roof": "Bamboo/Timber Heavy roof",
    "Reinforced Brick Slab/rcc/rbc": "rcc/rb/rbc",
}

train_cp["type_of_roof"] = train_cp["type_of_roof"].replace(type_of_roof_dict).str.strip()

train_cp.head()

,floors_before_eq (total),old_building,plinth_area (ft^2),height_before_eq (ft),land_surface_condition,type_of_foundation,type_of_roof,type_of_ground_floor,type_of_other_floor,position,...,type_of_reinforcement_concrete,residential_type,no_family_residing,public_place_type,industrial_use_type,govermental_use_type,flexible_superstructure,wall_binding,wall_material,damage_grade
0,1.0,22.0,300.0,10.0,1,Clay Sand Mixed mortar-Stone/Brick,Bamboo or Timber Light roof,mud,Not applicable,Not attached,...,0.0,Non-residential,None,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
1,3.0,22.0,360.0,18.0,2,Clay mortar-Stone/Brick,Wood Light Roof or Bamboo Heavy Roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,4.0
2,2.0,8.0,380.0,17.0,2,Clay mortar-Stone/Brick,Wood Light Roof or Bamboo Light Roof,Brick or Stone,Timber-Planck,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
3,2.0,22.0,250.0,14.0,2,Clay mortar-Stone/Brick,Bamboo or Timber Light roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,None,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
4,2.0,7.0,312.0,13.0,2,Clay Sand Mixed mortar-Stone/Brick,Bamboo or Timber Light roof,Clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,available,5.0,2.0,2.0


In [12]:
type_of_ground_floor_dict = {
    "rc": "reinforced concrete",
    "brick/stone": "brick or stone",
    "lumber": "wood",
    "timber": "wood",
    "mud": "clay"
}

train_cp["type_of_ground_floor"] = train_cp["type_of_ground_floor"].str.lower()\
    .replace(type_of_ground_floor_dict)\

train_cp.head()

,floors_before_eq (total),old_building,plinth_area (ft^2),height_before_eq (ft),land_surface_condition,type_of_foundation,type_of_roof,type_of_ground_floor,type_of_other_floor,position,...,type_of_reinforcement_concrete,residential_type,no_family_residing,public_place_type,industrial_use_type,govermental_use_type,flexible_superstructure,wall_binding,wall_material,damage_grade
0,1.0,22.0,300.0,10.0,1,Clay Sand Mixed mortar-Stone/Brick,Bamboo or Timber Light roof,clay,Not applicable,Not attached,...,0.0,Non-residential,None,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
1,3.0,22.0,360.0,18.0,2,Clay mortar-Stone/Brick,Wood Light Roof or Bamboo Heavy Roof,clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,4.0
2,2.0,8.0,380.0,17.0,2,Clay mortar-Stone/Brick,Wood Light Roof or Bamboo Light Roof,brick or stone,Timber-Planck,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
3,2.0,22.0,250.0,14.0,2,Clay mortar-Stone/Brick,Bamboo or Timber Light roof,clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,None,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
4,2.0,7.0,312.0,13.0,2,Clay Sand Mixed mortar-Stone/Brick,Bamboo or Timber Light roof,clay,TImber/Bamboo-Mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,available,5.0,2.0,2.0


In [13]:
type_of_other_floor_dict = {
    "lumber-plank": "wood-plank",
    "timber-planck": "wood-plank",
    "timber/bamboo-mud": "wood-mud or bamboo mud",
    "reinforced cement concrete/rb/rbc": "rcc/rb/rbc",
    "wood or bamboo mud": "wood-mud or bamboo mud",
    "timber mud or bamboo-mud": "wood-mud or bamboo mud"
}

train_cp["type_of_other_floor"] = train_cp["type_of_other_floor"]\
    .str.lower()\
    .replace(type_of_other_floor_dict)

train_cp["type_of_other_floor"].value_counts()


wood-mud or bamboo mud               30033
wood-plank                            7578
not applicable                        7197
rcc/rb/rbc                            1917
reinforced brick concrete/rcc/rbc       76
Name: type_of_other_floor, dtype: int64

In [14]:
train_cp['no_family_residing'] = train_cp['no_family_residing'].replace(
    'None', '0.0').astype('float')

train_cp['no_family_residing'].value_counts()

1.0    39405
0.0     4436
2.0     2545
3.0      316
4.0       76
5.0       16
6.0        4
7.0        2
8.0        1
Name: no_family_residing, dtype: int64

In [15]:
train_cp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46801 entries, 0 to 46800
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   floors_before_eq (total)        46801 non-null  float64
 1   old_building                    46801 non-null  float64
 2   plinth_area (ft^2)              46801 non-null  float64
 3   height_before_eq (ft)           46801 non-null  float64
 4   land_surface_condition          46801 non-null  int64  
 5   type_of_foundation              46801 non-null  object 
 6   type_of_roof                    46801 non-null  object 
 7   type_of_ground_floor            46801 non-null  object 
 8   type_of_other_floor             46801 non-null  object 
 9   position                        46801 non-null  object 
 10  building_plan_configuration     46801 non-null  object 
 11  technical_solution_proposed     46801 non-null  object 
 12  legal_ownership_status          

In [16]:
train_cp["residential_type"] = train_cp["residential_type"].replace({
    "Other Residential Type": "Other"
})

train_cp["residential_type"].value_counts()

Non-residential       44591
Hotel/Motel            1640
Rental Residential      368
Housing                 180
Other                    22
Name: residential_type, dtype: int64

In [17]:
train_cp['govermental_use_type'] = train_cp['govermental_use_type'].replace(
    ['Police Offices'], 'Govermental Buildings')

In [18]:
train_cp["public_place_type"] = train_cp["public_place_type"]\
    .apply(lambda x: "Public" if x != "Non-public" else "Non-public")

In [19]:
legal_ownership_status_dict = {
    "Private": "Private",
    "Private Use": "Private",
    "Prvt": "Private",
    "Privste": "Private",
    "Public Use": "Public",
    "Public Space": "Public",
    "Institutional Use": "Institutional",
    "Institutionals": "Institutional",
    "Unknown": "Other",
    "Unspecified": "Other",
}

train_cp["legal_ownership_status"] = train_cp["legal_ownership_status"].str.strip().replace(legal_ownership_status_dict)
train_cp["legal_ownership_status"].value_counts()

Private          44824
Public            1244
Institutional      490
Other              243
Name: legal_ownership_status, dtype: int64

In [20]:
train_cp.to_csv("../dataset/processed/train_cleaned.csv", index=False)

In [21]:
train_cp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46801 entries, 0 to 46800
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   floors_before_eq (total)        46801 non-null  float64
 1   old_building                    46801 non-null  float64
 2   plinth_area (ft^2)              46801 non-null  float64
 3   height_before_eq (ft)           46801 non-null  float64
 4   land_surface_condition          46801 non-null  int64  
 5   type_of_foundation              46801 non-null  object 
 6   type_of_roof                    46801 non-null  object 
 7   type_of_ground_floor            46801 non-null  object 
 8   type_of_other_floor             46801 non-null  object 
 9   position                        46801 non-null  object 
 10  building_plan_configuration     46801 non-null  object 
 11  technical_solution_proposed     46801 non-null  object 
 12  legal_ownership_status          

In [22]:
technical_solution_proposed_dict = {
    "Reconstruction": 0,
    "Major repair": 1,
    "Minor repair": 2,
    "No need": 3,
}

train_cp["technical_solution_proposed"] = train_cp["technical_solution_proposed"].replace(technical_solution_proposed_dict)
train_cp.head()

,floors_before_eq (total),old_building,plinth_area (ft^2),height_before_eq (ft),land_surface_condition,type_of_foundation,type_of_roof,type_of_ground_floor,type_of_other_floor,position,...,type_of_reinforcement_concrete,residential_type,no_family_residing,public_place_type,industrial_use_type,govermental_use_type,flexible_superstructure,wall_binding,wall_material,damage_grade
0,1.0,22.0,300.0,10.0,1,Clay Sand Mixed mortar-Stone/Brick,Bamboo or Timber Light roof,clay,not applicable,Not attached,...,0.0,Non-residential,0.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
1,3.0,22.0,360.0,18.0,2,Clay mortar-Stone/Brick,Wood Light Roof or Bamboo Heavy Roof,clay,wood-mud or bamboo mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,4.0
2,2.0,8.0,380.0,17.0,2,Clay mortar-Stone/Brick,Wood Light Roof or Bamboo Light Roof,brick or stone,wood-plank,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
3,2.0,22.0,250.0,14.0,2,Clay mortar-Stone/Brick,Bamboo or Timber Light roof,clay,wood-mud or bamboo mud,Not attached,...,0.0,Non-residential,0.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
4,2.0,7.0,312.0,13.0,2,Clay Sand Mixed mortar-Stone/Brick,Bamboo or Timber Light roof,clay,wood-mud or bamboo mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,available,5.0,2.0,2.0


In [23]:
flexible_superstructure_dict = {
    "unavailable": 0,
    "available": 1,
}

train_cp["flexible_superstructure"] = train_cp["flexible_superstructure"].replace(
    flexible_superstructure_dict)


In [24]:
public_place_type_dict = {
    "Public": 0,
    "Non-public": 1,
}

train_cp["public_place_type"] = train_cp["public_place_type"].replace(public_place_type_dict)

In [25]:
governmental_use_type_dict = {
    "Govermental Buildings": 0,
    "Non-govermental": 1,
}

train_cp["govermental_use_type"]  = train_cp["govermental_use_type"].replace(
    governmental_use_type_dict)

train_cp["govermental_use_type"].value_counts()


1    46786
0       15
Name: govermental_use_type, dtype: int64

In [26]:
ohe_col = list(train_cp.select_dtypes(include=['object']).columns)
ohe_col

['type_of_foundation',
 'type_of_roof',
 'type_of_ground_floor',
 'type_of_other_floor',
 'position',
 'building_plan_configuration',
 'legal_ownership_status',
 'residential_type',
 'industrial_use_type']

In [33]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

ohe_df = pd.DataFrame(ohe.fit_transform(train_cp[ohe_col]))
ohe_df.columns = ohe.get_feature_names(ohe_col)

train_ohe = train_cp.copy() 
for cat in ohe_col:
    train_ohe.drop(cat, axis=1, inplace=True)

print(train_ohe.shape)

train_ohe_merge = pd.concat([train_ohe, ohe_df], axis=1)

print(train_ohe_merge.shape)


(46801, 15)
(46801, 69)


In [34]:
from sklearn.preprocessing import RobustScaler

s = RobustScaler()



train_ohe_merge[["old_building",
                 "plinth_area (ft^2)", "height_before_eq (ft)"]] = s.fit_transform(train_ohe_merge[["old_building", "plinth_area (ft^2)", "height_before_eq (ft)"]])

train_ohe_merge.head()

,floors_before_eq (total),old_building,plinth_area (ft^2),height_before_eq (ft),land_surface_condition,technical_solution_proposed,has_secondary_use,type_of_reinforcement_concrete,no_family_residing,public_place_type,...,residential_type_Rental Residential,industrial_use_type_Agro,industrial_use_type_Farm,industrial_use_type_Food,industrial_use_type_Forest-based,industrial_use_type_Infrastructure,industrial_use_type_Manufacturing,industrial_use_type_Metallurgy,industrial_use_type_Non-industrial,industrial_use_type_Service/Tourism
0,1.0,0.333333,-0.319149,-1.000000,1,0,0.0,0.0,0.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,3.0,0.333333,0.000000,0.333333,2,0,0.0,0.0,1.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2.0,-0.444444,0.106383,0.166667,2,0,0.0,0.0,1.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2.0,0.333333,-0.585106,-0.333333,2,0,0.0,0.0,0.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2.0,-0.500000,-0.255319,-0.500000,2,1,0.0,0.0,1.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [35]:
train_ohe_merge["damage_grade"] = train_ohe_merge["damage_grade"].astype('int')
train_ohe_merge["damage_grade"] = train_ohe_merge["damage_grade"] - 1
train_ohe_merge.to_csv("../dataset/processed/train_encoded.csv", index=False)

In [71]:
def clean(df):
    angka = {
        "one": 1,
        "two": 2,
        "three": 3,
        "four": 4,
        "five": 5,
        "six": 6,
        "seven": 7,
        "eight": 8,
        "nine": 9,
        "1st": 1,
        "2nd": 2,
        "3rd": 3,
        "fifth": 5,
        "second": 2,
        "third": 3,
        "3.00": 3,
    }

    delete = {
        "story": "",
        "stories": "",
        "floor": "",
        "floors": "",
        "has": "",
        "there is": "",
        "just": "",
        "-": "",
        "fl": "",
        "/": "",
    }

    df["floors_before_eq (total)"] = df["floors_before_eq (total)"]\
        .str.lower()\
        .replace(delete, regex=True)\
        .str.strip()\
        .replace(angka, regex=True)

    df["floors_before_eq (total)"] = df["floors_before_eq (total)"].astype(
        "float")
    df["plinth_area (ft^2)"] = df["plinth_area (ft^2)"].str.lower()

    df["plinth_area (ft^2)"] = df["plinth_area (ft^2)"].apply(
        lambda x: x.replace(" ft^2", ""))
    df["plinth_area (ft^2)"] = df["plinth_area (ft^2)"].apply(
        lambda x: x.replace("more than ", ""))
    df["plinth_area (ft^2)"] = df["plinth_area (ft^2)"].astype(
        "float")

    land_surface_condition_dict = {
        "Steep slope": 0,
        "Moderate slope": 1,
        "Flat": 2,
    }

    df["land_surface_condition"] = df["land_surface_condition"].replace(
        land_surface_condition_dict)

    type_of_foundation_dict = {
        "Mud mortar-Stone/Brick": "Clay mortar-Stone/Brick",
        "Bamboo/TImber": "Bamboo or Timber",
        "Bamboo/Timber": "Bamboo or Timber",
        "RC": "Reinforced Concrete",
        "Others": "Other",
        "Cement-Stone or Cement-Brick": "Cement-Stone/Brick"
    }

    df["type_of_foundation"] = df["type_of_foundation"]\
        .replace(type_of_foundation_dict)\
        .str.strip()

    type_of_roof_dict = {
        "Bamboo/TImber-Heavy Roof": "Bamboo/Timber Heavy roof",
        "Bamboo/TImber-Light Roof": "Bamboo or Timber Light roof",
        "Bamboo/Timber Light roof": "Bamboo or Timber Light roof ",
        "reinforced cement concrete/rb/rbc": "rcc/rb/rbc",
        "Reinforced brick concrete/rcc/rbc": "rcc/rb/rbc",
        "Bamboo or Timber Heavy roof": "Bamboo/Timber Heavy roof",
        "Reinforced Brick Slab/rcc/rbc": "rcc/rb/rbc",
    }

    df["type_of_roof"] = df["type_of_roof"].replace(
        type_of_roof_dict).str.strip()

    type_of_ground_floor_dict = {
        "rc": "reinforced concrete",
        "brick/stone": "brick or stone",
        "lumber": "wood",
        "timber": "wood",
        "mud": "clay"
    }

    df["type_of_ground_floor"] = df["type_of_ground_floor"].str.lower()\
        .replace(type_of_ground_floor_dict)\

    df.head()

    type_of_other_floor_dict = {
        "lumber-plank": "wood-plank",
        "timber-planck": "wood-plank",
        "timber/bamboo-mud": "wood-mud or bamboo mud",
        "reinforced cement concrete/rb/rbc": "rcc/rb/rbc",
        "wood or bamboo mud": "wood-mud or bamboo mud",
        "timber mud or bamboo-mud": "wood-mud or bamboo mud"
    }

    df["type_of_other_floor"] = df["type_of_other_floor"]\
        .str.lower()\
        .replace(type_of_other_floor_dict)

    df['no_family_residing'] = df['no_family_residing'].replace(
        'None', '0.0').astype('float')

    df["residential_type"] = df["residential_type"].replace({
        "Other Residential Type": "Other"
    })
    df['govermental_use_type'] = df['govermental_use_type'].replace(
        ['Police Offices'], 'Govermental Buildings')
    df["public_place_type"] = df["public_place_type"]\
        .apply(lambda x: "Public" if x != "Non-public" else "Non-public")

    legal_ownership_status_dict = {
        "Private": "Private",
        "Private Use": "Private",
        "Prvt": "Private",
        "Privste": "Private",
        "Public Use": "Public",
        "Public Space": "Public",
        "Institutional Use": "Institutional",
        "Institutionals": "Institutional",
        "Unknown": "Other",
        "Unspecified": "Other",
    }

    df["legal_ownership_status"] = df["legal_ownership_status"].str.strip(
    ).replace(legal_ownership_status_dict)

    technical_solution_proposed_dict = {
        "Reconstruction": 0,
        "Major repair": 1,
        "Minor repair": 2,
        "No need": 3,
    }

    df["technical_solution_proposed"] = df["technical_solution_proposed"].replace(
        technical_solution_proposed_dict)

    flexible_superstructure_dict = {
        "unavailable": 0,
        "available": 1,
    }

    df["flexible_superstructure"] = df["flexible_superstructure"].replace(
        flexible_superstructure_dict)

    public_place_type_dict = {
        "Public": 0,
        "Non-public": 1,
    }

    df["public_place_type"] = df["public_place_type"].replace(
        public_place_type_dict)

    governmental_use_type_dict = {
        "Govermental Buildings": 0,
        "Non-govermental": 1,
    }

    df["govermental_use_type"] = df["govermental_use_type"].replace(
        governmental_use_type_dict)

    ohe_col = list(df.select_dtypes(include=['object']).columns)

    ohe_df = ohe.transform(df[ohe_col])
    ohe_df = pd.DataFrame(ohe_df, columns=ohe.get_feature_names(ohe_col))
    train_ohe = df.copy()
    for cat in ohe_col:
        train_ohe.drop(cat, axis=1, inplace=True)
    train_ohe_merge = pd.concat([train_ohe, ohe_df], axis=1)
    train_ohe_merge[["old_building",
                     "plinth_area (ft^2)", "height_before_eq (ft)"]] = s.transform(train_ohe_merge[["old_building", "plinth_area (ft^2)", "height_before_eq (ft)"]])

    import re
    train_ohe_merge = train_ohe_merge.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))

    return train_ohe_merge


In [41]:
import re
df = train_ohe_merge.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))

In [50]:
from sklearn.model_selection import train_test_split

X = df.drop(["damage_grade"], axis=1)
y = df["damage_grade"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=69420)

print(f"""\
X_train shape: {X_train.shape}
X_test shape: {X_test.shape}
y_train shape: {y_train.shape}
y_test shape: {y_test.shape} 
""")

X_train shape: (35100, 68)
X_test shape: (11701, 68)
y_train shape: (35100,)
y_test shape: (11701,) 



In [53]:
from sklearn.metrics import f1_score
import xgboost as xgb

params = {
            'lambda': 0.0026651196200513246, 
            'alpha': 0.02762188645863784, 
            'colsample_bytree': 0.9,
            'subsample': 0.7, 
            'learning_rate': 0.02, 
            'n_estimators': 8244, 
            'max_depth': 11, 
            'min_child_weight': 127
          }

xgb_model = xgb.XGBClassifier(**params)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

print(f1_score(y_test, y_pred, average='macro'))

0.6677997121591929


In [63]:
xgb_mini = xgb.XGBClassifier()
xgb_mini.fit(X_train, y_train)

y_pred = xgb_mini.predict(X_test)

print(f1_score(y_test, y_pred, average='macro'))


0.657549910525127


In [72]:
test = pd.read_csv("../dataset/raw/test.csv")
test_clean = clean(test)

test.head()

,id,floors_before_eq (total),old_building,plinth_area (ft^2),height_before_eq (ft),land_surface_condition,type_of_foundation,type_of_roof,type_of_ground_floor,type_of_other_floor,...,has_secondary_use,type_of_reinforcement_concrete,residential_type,no_family_residing,public_place_type,industrial_use_type,govermental_use_type,flexible_superstructure,wall_binding,wall_material
0,0,2.0,7,418.0,14,2,Clay mortar-Stone/Brick,Bamboo or Timber Light roof,clay,wood-mud or bamboo mud,...,0.0,0,Non-residential,1.0,1,Non-industrial,1,0,5,2
1,1,3.0,13,396.0,21,2,Clay mortar-Stone/Brick,Bamboo/Timber Heavy roof,brick or stone,wood-mud or bamboo mud,...,0.0,0,Non-residential,1.0,1,Non-industrial,1,0,5,2
2,2,3.0,40,400.0,18,2,Clay mortar-Stone/Brick,Bamboo or Timber Light roof,clay,wood-mud or bamboo mud,...,0.0,0,Non-residential,1.0,1,Non-industrial,1,0,5,2
3,3,2.0,25,378.0,20,2,Bamboo or Timber,Bamboo/Timber Heavy roof,clay,wood-mud or bamboo mud,...,0.0,0,Non-residential,1.0,1,Non-industrial,1,0,5,2
4,4,2.0,5,375.0,20,2,Clay mortar-Stone/Brick,Wood Light Roof or Bamboo Light Roof,clay,wood-mud or bamboo mud,...,0.0,0,Non-residential,1.0,1,Non-industrial,1,0,5,2


In [73]:
test_clean.to_csv("../dataset/processed/test_encoded.csv", index=False)

In [74]:
X_submit = test_clean.drop("id", axis=1)

y_sub = xgb_mini.predict(X_submit)
y_sub

array([4, 4, 4, ..., 0, 3, 4], dtype=int64)